## Presentation Types with Hidden Types
## Also Network Visualization with Louvain Communities

* March 2022 version

    * Uses getDistance to identify `close matches` with side-by-side comparison of soggetti.  With a distance of "1", the soggetti `4, 1, 2, 3`, and `5, 1, 2, 3` will count as the same.  These are reported as "flexed entries" in a separate column.

    * Labels Fuga, PEn, and ID according to time intervals.  
    * If two entries are separated by more than 10 bars (80 offsets), the tool resets to a new pattern
    * Finds time intervals between entries (expressed as offsets, like `8.0, 4.0, 8.0`)
    * Finds melodic intervals between first note of successive entries in each pattern (like `P-5, P-8`)
    * Counts number of entries
    * Provides offset and measure/beat locations
    * Sorts all presentation types by the order in which they appear in the piece
    * Reports voice names of the entries, in order of their appearance
    * Omits singleton soggetti (just one entry of a given motive in isolation)
    
    ALSO
    
    * Finds "hidden" types within a longer Fuga.  That is, if a 5-voice fuga also contains a PEN, it will label both of these as separate presentation type, along with all the relevant data noted above.

In [1]:
import intervals
from intervals import * 
from intervals import main_objs
import intervals.visualizations as viz
import pandas as pd
import re
import altair as alt 
from ipywidgets import interact
from pandas.io.json import json_normalize
from pyvis.network import Network
from IPython.display import display
import requests
import os
import numpy
import itertools
from itertools import combinations
import networkx as nx
from community import community_louvain
from copy import deepcopy
MYDIR = ("saved_csv")
CHECK_FOLDER = os.path.isdir(MYDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MYDIR)
    print("created folder : ", MYDIR)

else:
    print(MYDIR, "folder already exists.")

saved_csv folder already exists.


#### The following are special functions used by the classifier.  Don't change them.

In [138]:
def find_entry_int_distance(coordinates, piece: intervals.main_objs.ImportedPiece):
    
    """
    This function finds the melodic intervals between the first notes of 
    successive entries in a given presentation type.  
    They are represented as intervals with quality and direction, thus P-4, m3, P5, P5, M-9, P-4, P4
    
    """
    
    tone_list = []
    
    all_tones = piece.getNoteRest()
    
    for item in coordinates:
        filtered_tones = all_tones.loc[item] 
        tone_list.append(filtered_tones)
        
    noteObjects = [note.Note(tone) for tone in tone_list]
    _ints = [interval.Interval(noteObjects[i], noteObjects[i + 1]) for i in range(len(noteObjects) - 1)]
    entry_ints = []
    
    for _int in _ints:
        entry_ints.append(_int.directedName)
    
    return entry_ints

def split_by_threshold(seq, max_diff=70):  
    
    """
    This function finds gaps between sequences of matching melodic entries.  
    The threshold is set to 70 offsets by default--under about 10 measures.
    
    """
    it = iter(seq)
    last = next(it)
    part = [last]

    for curr in it:
        if curr - last > max_diff:
            yield part
            part = []

        part.append(curr)
        last = curr
#         print(part)
        
    yield part
    

def classify_by_offset(offset_diffs):
    """
    This function predicts the Presentation Types. It relies of the differences between 
    the first offsets of successive melodic entries. 
    
    If the offset differences are identical:  PEN
    If the odd-numbered offset differences are identical:  ID, since these represent
    situations in which the entries 1-2 have the same offset difference as entries 3-4
    If the offset differences are all different:  FUGA
    
    """
    alt_list = offset_diffs[::2]

    if len(set(offset_diffs)) == 1 and len(offset_diffs) > 1:
        return 'PEN'
    # elif (len(offset_difference_list) %2 != 0) and (len(set(alt_list)) == 1):
    elif (len(offset_diffs) % 2 != 0) and (len(set(alt_list)) == 1) and (len(offset_diffs) >= 3):
        return 'ID'
    elif len(offset_diffs) >= 1:
        return 'FUGA'

    

def temp_dict_of_details(slist, entry_array, det, matches):
    """
    This function assembles various features for the presentation types 
    into a single temporary dictionary, which in turn is appended to the dataframe of 'points'
    
    """
    
    array = entry_array[entry_array.index.get_level_values(0).isin(slist)]
    short_offset_list = array.index.to_list()
    time_ints = numpy.diff(array.index).tolist()
    voice_list = array['voice'].to_list()
    tone_coordinates =  list(zip(short_offset_list, voice_list))
    mel_ints = find_entry_int_distance(tone_coordinates, piece)
    first_offset = short_offset_list[0]
    meas_beat = det[det.index.get_level_values('Offset').isin(short_offset_list)]
    mb2 = meas_beat.reset_index()
    mb2['mb'] = mb2["Measure"].astype(str) + "/" + mb2["Beat"].astype(str)
    meas_beat_list = mb2['mb'].to_list()
    
    # temp results for this set
    temp = {"Composer": piece.metadata["composer"],
            "Title": piece.metadata["title"],
            'First_Offset': first_offset, 
            'Offsets': short_offset_list, 
            'Measures_Beats': meas_beat_list,
            "Soggetti": matches,
            'Voices': voice_list, 
            'Time_Entry_Intervals': time_ints, 
            'Melodic_Entry_Intervals': mel_ints}
    return temp

def classify_entries_as_presentation_types(piece):
    
    """
    This function uses several other functions to classify the entries in a given piece.
    The output is a list, in order of offset, of each presentation type, including information about
    measures/beats
    starting offset
    soggetti involved 
    melodic intervals of entry
    time intervals of entry
    
    
    """
    # Classifier with Functions
    points = pd.DataFrame()
    points2 = pd.DataFrame()
    # new_offset_list = []
    nr = piece.getNoteRest()
    det = piece.detailIndex(nr, offset=True)

    # durations and ngrams of durations
    dur = piece.getDuration(df=nr)
    dur_ng = piece.getNgrams(df=dur, n=3)

    # ngrams of melodic entries
    # for chromatic, use:
    # piece.getMelodicEntries(interval_settings=('c', True, True), n=5)
    mel = piece.getMelodicEntries(n=3)
    mels_stacked = mel.stack().to_frame()
    mels_stacked.rename(columns =  {0:"pattern"}, inplace = True)

    # edit distance, based on side-by-side comparison of melodic ngrams
    # gets flexed and other similar soggetti
    dist = piece.getDistance(mel)
    dist_stack = dist.stack().to_frame()


    # filter distances to threshold.  <2 is good
    filtered_dist_stack = dist_stack[dist_stack[0] < 2]
    filtered_dist = filtered_dist_stack.reset_index()
    filtered_dist.rename(columns =  {'level_0':"source", 'level_1':'match'}, inplace = True)

    # Group the filtered distanced patterns
    full_list_of_matches = filtered_dist.groupby('source')['match'].apply(list).reset_index()

    for matches in full_list_of_matches["match"]:
        related_entry_list = mels_stacked[mels_stacked['pattern'].isin(matches)]
        entry_array = related_entry_list.reset_index(level=1).rename(columns = {'level_1': "voice", 0: "pattern"})
        offset_list = entry_array.index.to_list()
        split_list = list(split_by_threshold(offset_list))
        # here is the list of starting offsets of the original set of entries:  slist
        slist = split_list[0]
        temp = temp_dict_of_details(slist, entry_array, det, matches)

        points = points.append(temp, ignore_index=True)
        points['Presentation_Type'] = points['Time_Entry_Intervals'].apply(classify_by_offset)
        points.drop_duplicates(subset=["First_Offset"], keep='first', inplace = True)
        points = points[points['Offsets'].apply(len) > 1]

        l = len(slist)
        m = l + 1
        if l > 2:
            for r in range(3, 6):
                list_combinations = list(combinations(slist, r))
                for tiny_list in list_combinations:

                    temp = temp_dict_of_details(tiny_list, entry_array, det, matches)

                    temp["Presentation_Type"] = classify_by_offset(temp['Time_Entry_Intervals'])

                    if 'PEN' in temp["Presentation_Type"]:
                        points2 = points2.append(temp, ignore_index=True)#.sort_values("First_Offset")

                    if 'ID' in temp["Presentation_Type"]:
                        points2 = points2.append(temp, ignore_index=True)#.sort_values("First_Offset"
                    
# # HERE WE NEED TO FIND A WAY TO SEARCH ONLY THE UNIQUE COMBINATIONS!

# # MUST DROP DUPLICATES ON THE BASIS OF THE OFFSET_LIST, NOT THE FIRST OFFSET!
# # THE OFFSET LIST IS A "LIST" AND WE MUST CHANGE THAT INTO A STRING IN ORDER TO DROP DUPLICATES


    points_combined = points.append(points2, ignore_index=True).sort_values("First_Offset").reset_index(drop=True)
    points_combined["Offsets_Key"] = points_combined["Offsets"].apply(offset_joiner)
    points_combined.drop_duplicates(subset=["Offsets_Key"], keep='first', inplace = True)
    points_combined['Flexed_Entries'] = points_combined["Soggetti"].apply(len) > 1
    points_combined["Number_Entries"] = points_combined["Offsets"].apply(len) 
    col_order = ['Composer', 
             'Title', 
             'First_Offset', 
             'Measures_Beats', 
             'Melodic_Entry_Intervals',
             'Offsets', 
             'Soggetti', 
             'Time_Entry_Intervals', 
             'Voices',
             'Presentation_Type', 
              'Number_Entries',
            'Flexed_Entries']
    points_combined = points_combined.reindex(columns=col_order)
    return points_combined


# #  the following are used to turn the offset diffs and melodic entry intervals 
# # and melodies into strings for the network 


def joiner(a):
    b = '_'.join(map(str, a))
    return b
def clean_melody(c):  
    d = (('_').join(c[0].split(","))).replace(" ", "")
    return d
def offset_joiner(a):
    b = '_'.join(map(str, a))
    return b

## Load one Piece Here

* Note that you can load from CRIM, or put a file in the **Music_Files** folder in the Notebook.

In [26]:
# piece = importScore('Music_Files/Senfl_Ave_forCRIM.mei_msg.mei')
piece = importScore('https://crimproject.org/mei/CRIM_Mass_0013_1.mei')
# piece = importScore('Music_Files/CRIM_Mass_0007_4.mei')


Successfully imported https://crimproject.org/mei/CRIM_Mass_0013_1.mei


## Run the Classifier Here.  



In [140]:
output = classify_entries_as_presentation_types(piece)
len(output)

122

In [144]:
output.to_csv("saved_csv/Mass_0013_1_PTs.csv")

In [145]:
output


,Composer,Title,First_Offset,Measures_Beats,Melodic_Entry_Intervals,Offsets,Soggetti,Time_Entry_Intervals,Voices,Presentation_Type,Number_Entries,Flexed_Entries
0,Pierre de Manchicourt,Missa Quo abiit dilectus tuus: Kyrie,0.0,"[1/1.0, 3/1.0, 6/3.0, 8/3.0]","[P-4, P-5, P-4]","[0.0, 16.0, 44.0, 60.0]","[5, -2, 2, 4, -2, 2]","[16.0, 28.0, 16.0]","[Superius, Contratenor, Tenor, Bassus]",ID,4,True
3,Pierre de Manchicourt,Missa Quo abiit dilectus tuus: Kyrie,16.0,"[3/1.0, 8/3.0]",[P-8],"[16.0, 60.0]","[4, -2, 2, 3, -2, 2]",[44.0],"[Contratenor, Bassus]",FUGA,2,True
4,Pierre de Manchicourt,Missa Quo abiit dilectus tuus: Kyrie,38.0,"[5/4.0, 11/2.0, 13/2.0, 18/4.0]","[P-8, P-5, P5]","[38.0, 82.0, 98.0, 142.0]","[-3, 2, -2, -3, 2, -3]","[44.0, 16.0, 44.0]","[Superius, Tenor, Bassus, Bassus]",ID,4,True
5,Pierre de Manchicourt,Missa Quo abiit dilectus tuus: Kyrie,38.0,"[5/4.0, 12/2.0, 15/2.0, 21/4.0]","[P1, P-8, P8]","[38.0, 90.0, 114.0, 166.0]","[-3, 2, -2, -3, 2, -3]","[52.0, 24.0, 52.0]","[Superius, Superius, Tenor, Superius]",ID,4,True
6,Pierre de Manchicourt,Missa Quo abiit dilectus tuus: Kyrie,38.0,"[5/4.0, 13/4.0, 21/4.0]","[P-4, P4]","[38.0, 102.0, 166.0]","[-3, 2, -2, -3, 2, -3]","[64.0, 64.0]","[Superius, Contratenor, Superius]",PEN,3,True
7,Pierre de Manchicourt,Missa Quo abiit dilectus tuus: Kyrie,38.0,"[5/4.0, 13/2.0, 20/4.0]","[P-12, P5]","[38.0, 98.0, 158.0]","[-3, 2, -2, -3, 2, -3]","[60.0, 60.0]","[Superius, Bassus, Tenor]",PEN,3,True
8,Pierre de Manchicourt,Missa Quo abiit dilectus tuus: Kyrie,38.0,"[5/4.0, 12/2.0, 18/4.0]","[P1, P-8]","[38.0, 90.0, 142.0]","[-3, 2, -2, -3, 2, -3]","[52.0, 52.0]","[Superius, Superius, Bassus]",PEN,3,True
9,Pierre de Manchicourt,Missa Quo abiit dilectus tuus: Kyrie,38.0,"[5/4.0, 11/2.0, 13/4.0, 19/2.0]","[P-8, P5, M-2]","[38.0, 82.0, 102.0, 146.0]","[-3, 2, -2, -3, 2, -3]","[44.0, 20.0, 44.0]","[Superius, Tenor, Contratenor, Contratenor]",ID,4,True
11,Pierre de Manchicourt,Missa Quo abiit dilectus tuus: Kyrie,38.0,"[5/4.0, 11/2.0, 15/2.0, 20/4.0]","[P-8, P1, P1]","[38.0, 82.0, 114.0, 158.0]","[-3, 2, -2, -3, 2, -3]","[44.0, 32.0, 44.0]","[Superius, Tenor, Tenor, Tenor]",ID,4,True
16,Pierre de Manchicourt,Missa Quo abiit dilectus tuus: Kyrie,38.0,"[5/4.0, 11/2.0, 12/2.0, 13/2.0, 13/4.0, 15/2.0...","[P-8, P8, P-12, M9, P-5, P8, P-8, P4, P-4, P8,...","[38.0, 82.0, 90.0, 98.0, 102.0, 114.0, 130.0, ...","[-3, 2, -2, -3, 2, -3]","[44.0, 8.0, 8.0, 4.0, 12.0, 16.0, 12.0, 4.0, 1...","[Superius, Tenor, Superius, Bassus, Contrateno...",FUGA,12,True


In [44]:
def offset_joiner(a):
    b = '_'.join(map(str, a))
    return b


In [71]:
output2["Offsets_Key"] = output2["Offsets"].apply(offset_joiner)
# type(output2["Offsets_Key"][0])

In [69]:
len(output2)

89

In [88]:
pd.set_option('display.max_rows', 500) 

output2 = output
output2.drop_duplicates(subset=["Offsets_Key"], keep='first', inplace = True)

output2.reset_index(drop=True)
# len(output2)
output2["long_id"] = output2["Time_Entry_Intervals"][1] < 20
# output2["Time_Entry_Intervals"][0][1]
# output2[output2["long_id"] == True]

KeyError: 1

### Run Classifier on Several Pieces at Once

In [ ]:
piece_list = ['https://crimproject.org/mei/CRIM_Model_0008.mei',
             'https://crimproject.org/mei/CRIM_Mass_0005_1.mei',
             'https://crimproject.org/mei/CRIM_Mass_0005_2.mei',
             'https://crimproject.org/mei/CRIM_Mass_0005_3.mei',
             'https://crimproject.org/mei/CRIM_Mass_0005_4.mei',
             'https://crimproject.org/mei/CRIM_Mass_0005_5.mei',
             'https://crimproject.org/mei/CRIM_Model_0001.mei',
             'https://crimproject.org/mei/CRIM_Mass_0002_1.mei',
             'https://crimproject.org/mei/CRIM_Mass_0002_2.mei',
             'https://crimproject.org/mei/CRIM_Mass_0002_3.mei',
             'https://crimproject.org/mei/CRIM_Mass_0002_4.mei',
             'https://crimproject.org/mei/CRIM_Mass_0002_5.mei',
             'https://crimproject.org/mei/CRIM_Model_0015.mei',
             'https://crimproject.org/mei/CRIM_Mass_0013_1.mei',
             'https://crimproject.org/mei/CRIM_Mass_0013_2.mei',
             'https://crimproject.org/mei/CRIM_Mass_0013_3.mei',
             'https://crimproject.org/mei/CRIM_Mass_0013_4.mei',
             'https://crimproject.org/mei/CRIM_Mass_0013_5.mei',
             'https://crimproject.org/mei/CRIM_Model_0019.mei',
             'https://crimproject.org/mei/CRIM_Mass_0019_1.mei',
             'https://crimproject.org/mei/CRIM_Mass_0019_2.mei',
             'https://crimproject.org/mei/CRIM_Mass_0019_3.mei',
             'https://crimproject.org/mei/CRIM_Mass_0019_4.mei',
             'https://crimproject.org/mei/CRIM_Mass_0019_5.mei']



In [9]:
piece_list = ['https://crimproject.org/mei/CRIM_Model_0019.mei',
             'https://crimproject.org/mei/CRIM_Mass_0019_1.mei',
             'https://crimproject.org/mei/CRIM_Mass_0019_2.mei',
             'https://crimproject.org/mei/CRIM_Mass_0019_3.mei',
             'https://crimproject.org/mei/CRIM_Mass_0019_4.mei',
             'https://crimproject.org/mei/CRIM_Mass_0019_5.mei']

In [10]:
final = pd.DataFrame()
for work in piece_list:
    piece = importScore(work)
    output = classify_entries_as_presentation_types(piece)
    final = final.append(output, ignore_index=True)
final

Memoized piece detected.
Successfully imported https://crimproject.org/mei/CRIM_Mass_0019_1.mei
Successfully imported https://crimproject.org/mei/CRIM_Mass_0019_2.mei
Successfully imported https://crimproject.org/mei/CRIM_Mass_0019_3.mei
Successfully imported https://crimproject.org/mei/CRIM_Mass_0019_4.mei
Successfully imported https://crimproject.org/mei/CRIM_Mass_0019_5.mei


,Composer,Title,First_Offset,Measures_Beats,Melodic_Entry_Intervals,Offsets,Soggetti,Time_Entry_Intervals,Voices,Presentation_Type,Number_Entries,Flexed_Entries
0,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,0.0,"[1/1.0, 2/1.0, 6/1.0, 7/1.0, 11/3.0, 14/1.0, 1...","[P-5, P-4, P-5, P8, P5, P-8, P1, P8, P-5, P-4,...","[0.0, 8.0, 40.0, 48.0, 84.0, 104.0, 128.0, 150...","[-3, 3, 2, -2, -3, 2, 2, -2]","[8.0, 32.0, 8.0, 36.0, 20.0, 24.0, 22.0, 16.0,...","[Cantus, Altus, Tenor, Bassus, Altus, Cantus, ...",FUGA,15,True
1,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,0.0,"[1/1.0, 2/1.0, 6/1.0, 7/1.0]","[P-5, P-4, P-5]","[0.0, 8.0, 40.0, 48.0]","[-3, 3, 2, -2, -3, 2, 2, -2]","[8.0, 32.0, 8.0]","[Cantus, Altus, Tenor, Bassus]",ID,4,True
2,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,8.0,"[2/1.0, 6/1.0, 21/4.0, 25/4.0]","[P-4, P8, P-8]","[8.0, 40.0, 166.0, 198.0]","[-3, 3, 2, -2, -3, 2, 2, -2]","[32.0, 126.0, 32.0]","[Altus, Tenor, Cantus, Tenor]",ID,4,True
3,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,40.0,"[6/1.0, 11/3.0, 17/1.0]","[P4, P-4]","[40.0, 84.0, 128.0]","[-3, 3, 2, -2, -3, 2, 2, -2]","[44.0, 44.0]","[Tenor, Altus, Bassus]",PEN,3,True
4,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,48.0,"[7/1.0, 11/3.0, 27/4.0, 32/2.0]","[P8, P-8, P15]","[48.0, 84.0, 214.0, 250.0]","[-3, 3, 2, -2, -3, 2, 2, -2]","[36.0, 130.0, 36.0]","[Bassus, Altus, Bassus, Cantus]",ID,4,True
...,...,...,...,...,...,...,...,...,...,...,...,...
109,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Agnus Dei,404.0,"[50/3.0, 53/3.0]",[P-8],"[404.0, 428.0]","[1, 1, -3, 3]",[24.0],"[Cantus, QuintaPars]",FUGA,2,False
110,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Agnus Dei,446.0,"[55/4.0, 56/2.0, 56/4.0, 59/2.0, 59/4.0, 61/4....","[P-4, P4, P-11, P4, P11, P-8]","[446.0, 450.0, 454.0, 474.0, 478.0, 494.0, 518.0]","[-2, 2, 1, 3, -2, 2, 1, 2]","[4.0, 4.0, 20.0, 4.0, 16.0, 24.0]","[Altus, Tenor, Cantus, Bassus, QuintaPars, Can...",FUGA,7,True
111,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Agnus Dei,446.0,"[55/4.0, 56/2.0, 56/4.0]","[P-4, P4]","[446.0, 450.0, 454.0]","[-2, 2, 1, 3, -2, 2, 1, 2]","[4.0, 4.0]","[Altus, Tenor, Cantus]",PEN,3,True
112,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Agnus Dei,450.0,"[56/2.0, 56/4.0, 59/2.0, 59/4.0]","[P4, P-11, P4]","[450.0, 454.0, 474.0, 478.0]","[-2, 2, 1, 3, -2, 2, 1, 2]","[4.0, 20.0, 4.0]","[Tenor, Cantus, Bassus, QuintaPars]",ID,4,True


In [11]:
final

,Composer,Title,First_Offset,Measures_Beats,Melodic_Entry_Intervals,Offsets,Soggetti,Time_Entry_Intervals,Voices,Presentation_Type,Number_Entries,Flexed_Entries
0,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,0.0,"[1/1.0, 2/1.0, 6/1.0, 7/1.0, 11/3.0, 14/1.0, 1...","[P-5, P-4, P-5, P8, P5, P-8, P1, P8, P-5, P-4,...","[0.0, 8.0, 40.0, 48.0, 84.0, 104.0, 128.0, 150...","[-3, 3, 2, -2, -3, 2, 2, -2]","[8.0, 32.0, 8.0, 36.0, 20.0, 24.0, 22.0, 16.0,...","[Cantus, Altus, Tenor, Bassus, Altus, Cantus, ...",FUGA,15,True
1,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,0.0,"[1/1.0, 2/1.0, 6/1.0, 7/1.0]","[P-5, P-4, P-5]","[0.0, 8.0, 40.0, 48.0]","[-3, 3, 2, -2, -3, 2, 2, -2]","[8.0, 32.0, 8.0]","[Cantus, Altus, Tenor, Bassus]",ID,4,True
2,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,8.0,"[2/1.0, 6/1.0, 21/4.0, 25/4.0]","[P-4, P8, P-8]","[8.0, 40.0, 166.0, 198.0]","[-3, 3, 2, -2, -3, 2, 2, -2]","[32.0, 126.0, 32.0]","[Altus, Tenor, Cantus, Tenor]",ID,4,True
3,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,40.0,"[6/1.0, 11/3.0, 17/1.0]","[P4, P-4]","[40.0, 84.0, 128.0]","[-3, 3, 2, -2, -3, 2, 2, -2]","[44.0, 44.0]","[Tenor, Altus, Bassus]",PEN,3,True
4,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,48.0,"[7/1.0, 11/3.0, 27/4.0, 32/2.0]","[P8, P-8, P15]","[48.0, 84.0, 214.0, 250.0]","[-3, 3, 2, -2, -3, 2, 2, -2]","[36.0, 130.0, 36.0]","[Bassus, Altus, Bassus, Cantus]",ID,4,True
...,...,...,...,...,...,...,...,...,...,...,...,...
109,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Agnus Dei,404.0,"[50/3.0, 53/3.0]",[P-8],"[404.0, 428.0]","[1, 1, -3, 3]",[24.0],"[Cantus, QuintaPars]",FUGA,2,False
110,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Agnus Dei,446.0,"[55/4.0, 56/2.0, 56/4.0, 59/2.0, 59/4.0, 61/4....","[P-4, P4, P-11, P4, P11, P-8]","[446.0, 450.0, 454.0, 474.0, 478.0, 494.0, 518.0]","[-2, 2, 1, 3, -2, 2, 1, 2]","[4.0, 4.0, 20.0, 4.0, 16.0, 24.0]","[Altus, Tenor, Cantus, Bassus, QuintaPars, Can...",FUGA,7,True
111,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Agnus Dei,446.0,"[55/4.0, 56/2.0, 56/4.0]","[P-4, P4]","[446.0, 450.0, 454.0]","[-2, 2, 1, 3, -2, 2, 1, 2]","[4.0, 4.0]","[Altus, Tenor, Cantus]",PEN,3,True
112,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Agnus Dei,450.0,"[56/2.0, 56/4.0, 59/2.0, 59/4.0]","[P4, P-11, P4]","[450.0, 454.0, 474.0, 478.0]","[-2, 2, 1, 3, -2, 2, 1, 2]","[4.0, 20.0, 4.0]","[Tenor, Cantus, Bassus, QuintaPars]",ID,4,True


In [12]:
final["MINT"] = final["Melodic_Entry_Intervals"].apply(joiner)
final["TINT"] = final["Time_Entry_Intervals"].apply(joiner)

final['SOG'] = final['Soggetti'].apply(clean_melody)
final['ALL'] = final["MINT"] + '_' + final["TINT"] + '_' + final['SOG']

final["ALL_INT"] = final["MINT"] + '_' + final["TINT"]
final["ALL_SOG"] = final["MINT"] + '_' + final["SOG"]
final

,Composer,Title,First_Offset,Measures_Beats,Melodic_Entry_Intervals,Offsets,Soggetti,Time_Entry_Intervals,Voices,Presentation_Type,Number_Entries,Flexed_Entries,MINT,TINT,SOG,ALL,ALL_INT,ALL_SOG
0,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,0.0,"[1/1.0, 2/1.0, 6/1.0, 7/1.0, 11/3.0, 14/1.0, 1...","[P-5, P-4, P-5, P8, P5, P-8, P1, P8, P-5, P-4,...","[0.0, 8.0, 40.0, 48.0, 84.0, 104.0, 128.0, 150...","[-3, 3, 2, -2, -3, 2, 2, -2]","[8.0, 32.0, 8.0, 36.0, 20.0, 24.0, 22.0, 16.0,...","[Cantus, Altus, Tenor, Bassus, Altus, Cantus, ...",FUGA,15,True,P-5_P-4_P-5_P8_P5_P-8_P1_P8_P-5_P-4_P-5_P8_P8_...,8.0_32.0_8.0_36.0_20.0_24.0_22.0_16.0_8.0_24.0...,-3_3_2_-2,P-5_P-4_P-5_P8_P5_P-8_P1_P8_P-5_P-4_P-5_P8_P8_...,P-5_P-4_P-5_P8_P5_P-8_P1_P8_P-5_P-4_P-5_P8_P8_...,P-5_P-4_P-5_P8_P5_P-8_P1_P8_P-5_P-4_P-5_P8_P8_...
1,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,0.0,"[1/1.0, 2/1.0, 6/1.0, 7/1.0]","[P-5, P-4, P-5]","[0.0, 8.0, 40.0, 48.0]","[-3, 3, 2, -2, -3, 2, 2, -2]","[8.0, 32.0, 8.0]","[Cantus, Altus, Tenor, Bassus]",ID,4,True,P-5_P-4_P-5,8.0_32.0_8.0,-3_3_2_-2,P-5_P-4_P-5_8.0_32.0_8.0_-3_3_2_-2,P-5_P-4_P-5_8.0_32.0_8.0,P-5_P-4_P-5_-3_3_2_-2
2,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,8.0,"[2/1.0, 6/1.0, 21/4.0, 25/4.0]","[P-4, P8, P-8]","[8.0, 40.0, 166.0, 198.0]","[-3, 3, 2, -2, -3, 2, 2, -2]","[32.0, 126.0, 32.0]","[Altus, Tenor, Cantus, Tenor]",ID,4,True,P-4_P8_P-8,32.0_126.0_32.0,-3_3_2_-2,P-4_P8_P-8_32.0_126.0_32.0_-3_3_2_-2,P-4_P8_P-8_32.0_126.0_32.0,P-4_P8_P-8_-3_3_2_-2
3,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,40.0,"[6/1.0, 11/3.0, 17/1.0]","[P4, P-4]","[40.0, 84.0, 128.0]","[-3, 3, 2, -2, -3, 2, 2, -2]","[44.0, 44.0]","[Tenor, Altus, Bassus]",PEN,3,True,P4_P-4,44.0_44.0,-3_3_2_-2,P4_P-4_44.0_44.0_-3_3_2_-2,P4_P-4_44.0_44.0,P4_P-4_-3_3_2_-2
4,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,48.0,"[7/1.0, 11/3.0, 27/4.0, 32/2.0]","[P8, P-8, P15]","[48.0, 84.0, 214.0, 250.0]","[-3, 3, 2, -2, -3, 2, 2, -2]","[36.0, 130.0, 36.0]","[Bassus, Altus, Bassus, Cantus]",ID,4,True,P8_P-8_P15,36.0_130.0_36.0,-3_3_2_-2,P8_P-8_P15_36.0_130.0_36.0_-3_3_2_-2,P8_P-8_P15_36.0_130.0_36.0,P8_P-8_P15_-3_3_2_-2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Agnus Dei,404.0,"[50/3.0, 53/3.0]",[P-8],"[404.0, 428.0]","[1, 1, -3, 3]",[24.0],"[Cantus, QuintaPars]",FUGA,2,False,P-8,24.0,1_1_-3_3,P-8_24.0_1_1_-3_3,P-8_24.0,P-8_1_1_-3_3
110,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Agnus Dei,446.0,"[55/4.0, 56/2.0, 56/4.0, 59/2.0, 59/4.0, 61/4....","[P-4, P4, P-11, P4, P11, P-8]","[446.0, 450.0, 454.0, 474.0, 478.0, 494.0, 518.0]","[-2, 2, 1, 3, -2, 2, 1, 2]","[4.0, 4.0, 20.0, 4.0, 16.0, 24.0]","[Altus, Tenor, Cantus, Bassus, QuintaPars, Can...",FUGA,7,True,P-4_P4_P-11_P4_P11_P-8,4.0_4.0_20.0_4.0_16.0_24.0,-2_2_1_3,P-4_P4_P-11_P4_P11_P-8_4.0_4.0_20.0_4.0_16.0_2...,P-4_P4_P-11_P4_P11_P-8_4.0_4.0_20.0_4.0_16.0_24.0,P-4_P4_P-11_P4_P11_P-8_-2_2_1_3
111,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Agnus Dei,446.0,"[55/4.0, 56/2.0, 56/4.0]","[P-4, P4]","[446.0, 450.0, 454.0]","[-2, 2, 1, 3, -2, 2, 1, 2]","[4.0, 4.0]","[Altus, Tenor, Cantus]",PEN,3,True,P-4_P4,4.0_4.0,-2_2_1_3,P-4_P4_4.0_4.0_-2_2_1_3,P-4_P4_4.0_4.0,P-4_P4_-2_2_1_3
112,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Agnus Dei,450.0,"[56/2.0, 56/4.0, 59/2.0, 59/4.0]","[P4, P-11, P4]","[450.0, 454.0, 474.0, 478.0]","[-2, 2, 1, 3, -2, 2, 1, 2]","[4.0, 20.0, 4.0]","[Tenor, Cantus, Bassus, QuintaPars]",ID,4,True,P4_P-11_P4,4.0_20.0_4.0,-2_2_1_3,P4_P-11_P4_4.0_20.0_4.0_-2_2_1_3,P4_P-11_P4_4.0_20.0_4.0,P4_P-11_P4_-2_2_1_3


In [13]:
filtered = final.loc[final['Number_Entries'] < 5] 
filtered

,Composer,Title,First_Offset,Measures_Beats,Melodic_Entry_Intervals,Offsets,Soggetti,Time_Entry_Intervals,Voices,Presentation_Type,Number_Entries,Flexed_Entries,MINT,TINT,SOG,ALL,ALL_INT,ALL_SOG
1,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,0.0,"[1/1.0, 2/1.0, 6/1.0, 7/1.0]","[P-5, P-4, P-5]","[0.0, 8.0, 40.0, 48.0]","[-3, 3, 2, -2, -3, 2, 2, -2]","[8.0, 32.0, 8.0]","[Cantus, Altus, Tenor, Bassus]",ID,4,True,P-5_P-4_P-5,8.0_32.0_8.0,-3_3_2_-2,P-5_P-4_P-5_8.0_32.0_8.0_-3_3_2_-2,P-5_P-4_P-5_8.0_32.0_8.0,P-5_P-4_P-5_-3_3_2_-2
2,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,8.0,"[2/1.0, 6/1.0, 21/4.0, 25/4.0]","[P-4, P8, P-8]","[8.0, 40.0, 166.0, 198.0]","[-3, 3, 2, -2, -3, 2, 2, -2]","[32.0, 126.0, 32.0]","[Altus, Tenor, Cantus, Tenor]",ID,4,True,P-4_P8_P-8,32.0_126.0_32.0,-3_3_2_-2,P-4_P8_P-8_32.0_126.0_32.0_-3_3_2_-2,P-4_P8_P-8_32.0_126.0_32.0,P-4_P8_P-8_-3_3_2_-2
3,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,40.0,"[6/1.0, 11/3.0, 17/1.0]","[P4, P-4]","[40.0, 84.0, 128.0]","[-3, 3, 2, -2, -3, 2, 2, -2]","[44.0, 44.0]","[Tenor, Altus, Bassus]",PEN,3,True,P4_P-4,44.0_44.0,-3_3_2_-2,P4_P-4_44.0_44.0_-3_3_2_-2,P4_P-4_44.0_44.0,P4_P-4_-3_3_2_-2
4,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,48.0,"[7/1.0, 11/3.0, 27/4.0, 32/2.0]","[P8, P-8, P15]","[48.0, 84.0, 214.0, 250.0]","[-3, 3, 2, -2, -3, 2, 2, -2]","[36.0, 130.0, 36.0]","[Bassus, Altus, Bassus, Cantus]",ID,4,True,P8_P-8_P15,36.0_130.0_36.0,-3_3_2_-2,P8_P-8_P15_36.0_130.0_36.0_-3_3_2_-2,P8_P-8_P15_36.0_130.0_36.0,P8_P-8_P15_-3_3_2_-2
5,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,84.0,"[11/3.0, 14/1.0, 29/4.0, 32/2.0]","[P5, P-5, P8]","[84.0, 104.0, 230.0, 250.0]","[-3, 3, 2, -2, -3, 2, 2, -2]","[20.0, 126.0, 20.0]","[Altus, Cantus, Altus, Cantus]",ID,4,True,P5_P-5_P8,20.0_126.0_20.0,-3_3_2_-2,P5_P-5_P8_20.0_126.0_20.0_-3_3_2_-2,P5_P-5_P8_20.0_126.0_20.0,P5_P-5_P8_-3_3_2_-2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Agnus Dei,202.0,"[26/2.0, 28/4.0, 31/2.0]","[M9, P4]","[202.0, 222.0, 242.0]","[1, -3, 3, -2]","[20.0, 20.0]","[Bassus, Tenor, Cantus]",PEN,3,False,M9_P4,20.0_20.0,1_-3_3_-2,M9_P4_20.0_20.0_1_-3_3_-2,M9_P4_20.0_20.0,M9_P4_1_-3_3_-2
109,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Agnus Dei,404.0,"[50/3.0, 53/3.0]",[P-8],"[404.0, 428.0]","[1, 1, -3, 3]",[24.0],"[Cantus, QuintaPars]",FUGA,2,False,P-8,24.0,1_1_-3_3,P-8_24.0_1_1_-3_3,P-8_24.0,P-8_1_1_-3_3
111,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Agnus Dei,446.0,"[55/4.0, 56/2.0, 56/4.0]","[P-4, P4]","[446.0, 450.0, 454.0]","[-2, 2, 1, 3, -2, 2, 1, 2]","[4.0, 4.0]","[Altus, Tenor, Cantus]",PEN,3,True,P-4_P4,4.0_4.0,-2_2_1_3,P-4_P4_4.0_4.0_-2_2_1_3,P-4_P4_4.0_4.0,P-4_P4_-2_2_1_3
112,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Agnus Dei,450.0,"[56/2.0, 56/4.0, 59/2.0, 59/4.0]","[P4, P-11, P4]","[450.0, 454.0, 474.0, 478.0]","[-2, 2, 1, 3, -2, 2, 1, 2]","[4.0, 20.0, 4.0]","[Tenor, Cantus, Bassus, QuintaPars]",ID,4,True,P4_P-11_P4,4.0_20.0_4.0,-2_2_1_3,P4_P-11_P4_4.0_20.0_4.0_-2_2_1_3,P4_P-11_P4_4.0_20.0_4.0,P4_P-11_P4_-2_2_1_3


### Network Visualization with Louvain Communities

In [14]:

f2 = filtered.groupby('MINT')['Title'].apply(list).reset_index()
f2


,MINT,Title
0,M-2_P5,[Missa Veni sponsa Christi: Sanctus]
1,M-9_P-5,[Missa Veni sponsa Christi: Credo]
2,M2_P-5,[Missa Veni sponsa Christi: Sanctus]
3,M9,[Missa Veni sponsa Christi: Credo]
4,M9_M-2,"[Missa Veni sponsa Christi: Gloria, Missa Veni..."
...,...,...
58,P8_P-8_P15,[Veni sponsa Christi]
59,P8_P1,[Missa Veni sponsa Christi: Sanctus]
60,P8_P5,[Missa Veni sponsa Christi: Sanctus]
61,m-3,[Missa Veni sponsa Christi: Credo]


In [15]:
pairs = f2.Title.apply(lambda x: list(combinations(x, 2)))
pairs

0                                                    []
1                                                    []
2                                                    []
3                                                    []
4     [(Missa Veni sponsa Christi: Gloria, Missa Ven...
                            ...                        
58                                                   []
59                                                   []
60                                                   []
61                                                   []
62                                                   []
Name: Title, Length: 63, dtype: object

In [16]:
pairs2 = pairs.explode().dropna()
pairs2

4     (Missa Veni sponsa Christi: Gloria, Missa Veni...
13    (Missa Veni sponsa Christi: Credo, Missa Veni ...
13    (Missa Veni sponsa Christi: Credo, Missa Veni ...
13    (Missa Veni sponsa Christi: Credo, Missa Veni ...
18    (Missa Veni sponsa Christi: Credo, Missa Veni ...
18    (Missa Veni sponsa Christi: Credo, Missa Veni ...
18    (Missa Veni sponsa Christi: Credo, Missa Veni ...
18    (Missa Veni sponsa Christi: Credo, Missa Veni ...
18    (Missa Veni sponsa Christi: Credo, Missa Veni ...
18    (Missa Veni sponsa Christi: Credo, Missa Veni ...
21    (Veni sponsa Christi, Missa Veni sponsa Christ...
21    (Veni sponsa Christi, Missa Veni sponsa Christ...
21    (Veni sponsa Christi, Missa Veni sponsa Christ...
21    (Missa Veni sponsa Christi: Kyrie, Missa Veni ...
21    (Missa Veni sponsa Christi: Kyrie, Missa Veni ...
21    (Missa Veni sponsa Christi: Kyrie, Missa Veni ...
25    (Veni sponsa Christi, Missa Veni sponsa Christ...
25    (Veni sponsa Christi, Missa Veni sponsa Ch

In [17]:
unique_pairs = pairs.explode().dropna().unique()
unique_pairs

array([('Missa Veni sponsa Christi: Gloria', 'Missa Veni sponsa Christi: Gloria'),
       ('Missa Veni sponsa Christi: Credo', 'Missa Veni sponsa Christi: Credo'),
       ('Missa Veni sponsa Christi: Credo', 'Missa Veni sponsa Christi: Sanctus'),
       ('Veni sponsa Christi', 'Missa Veni sponsa Christi: Kyrie'),
       ('Veni sponsa Christi', 'Missa Veni sponsa Christi: Sanctus'),
       ('Missa Veni sponsa Christi: Kyrie', 'Missa Veni sponsa Christi: Kyrie'),
       ('Missa Veni sponsa Christi: Kyrie', 'Missa Veni sponsa Christi: Sanctus'),
       ('Veni sponsa Christi', 'Missa Veni sponsa Christi: Credo'),
       ('Missa Veni sponsa Christi: Sanctus', 'Missa Veni sponsa Christi: Sanctus'),
       ('Missa Veni sponsa Christi: Kyrie', 'Missa Veni sponsa Christi: Credo'),
       ('Missa Veni sponsa Christi: Gloria', 'Missa Veni sponsa Christi: Agnus Dei'),
       ('Missa Veni sponsa Christi: Gloria', 'Missa Veni sponsa Christi: Sanctus'),
       ('Veni sponsa Christi', 'Missa Veni spon

In [18]:
pd.Series(unique_pairs).isna().sum()

0

In [19]:
def add_communities(G):
    G = deepcopy(G)
    partition = community_louvain.best_partition(G)
    nx.set_node_attributes(G, partition, "group")
    return G

In [20]:
pyvis_graph = Network(notebook=True, width="1800", height="1400", bgcolor="white", font_color="black")

In [22]:
G = nx.Graph()
G.add_edges_from(unique_pairs)
G = add_communities(G)

In [23]:
pyvis_graph.from_nx(G)

In [24]:
pyvis_graph.show('MINT.html')


#### Below is Development Work

In [ ]:
filtered = output.loc[output['Number_Entries'] < 4] 
filtered

In [ ]:
output = output.loc[output['Presentation_Type'] == "PEN"] 
output

In [ ]:
offset_diffs = [2.0, 1.0, 2.0, 3.0, 5.0, 6.0]
# some_list[start:stop:step]
alt_list = offset_diffs[::2]
alt_list

In [ ]:
# this works with ONE list of offsets

points2 = pd.DataFrame()
split_list = [90.0, 94.0, 102.0, 106.0, 134.0, 146.0, 162.0]

l = len(split_list)  
for r in range(3, l):
    list_combinations = list(combinations(split_list, r))
#             combo_time_ints = []
    for combo in list_combinations:
        combo_time_ints = numpy.diff(combo).tolist()
        combo_array = entry_array[entry_array.index.get_level_values(0).isin(combo)]
        combo_voice_list = combo_array['voice'].to_list()
        combo_patterns = combo_array['pattern']
        unique_combo_patterns = list(set(combo_patterns))
        tone_coordinates =  list(zip(combo, combo_voice_list))
# tone_coordinates.ffill(inplace=True)
        mel_ints = find_entry_int_distance(tone_coordinates, piece)
        hidden_type = classify_by_offset(combo_time_ints)

        meas_beat = det[det.index.get_level_values('Offset').isin(combo)]
        mb2 = meas_beat.reset_index()
        mb2['mb'] = mb2["Measure"].astype(str) + "/" + mb2["Beat"].astype(str)
        meas_beat_list = mb2['mb'].to_list()

        combo_temp = {'First_Offset': combo[0], 
            'Offsets': combo, 
            'Measures_Beats': meas_beat_list,
            'Presentation_Type': hidden_type,
            "Soggetti": unique_combo_patterns,
            'Voices': combo_voice_list, 
            'Time_Entry_Intervals': combo_time_ints, 
            'Melodic_Entry_Intervals': mel_ints}

        if 'PEN' in hidden_type:
            points2 = points2.append(combo_temp, ignore_index=True).sort_values("First_Offset")
#             points2 = points2[points2['Offsets'].apply(len) > 1]
        if 'ID' in hidden_type:
            points2 = points2.append(combo_temp, ignore_index=True).sort_values("First_Offset")
#             points2 = points2[points2['Offsets'].apply(len) > 1]
        
        
# combo_time_ints
# combo_array
# # combo_voice_list
# # combo_patterns
# # unique_combo_patterns
# # tone_coordinates
# # mel_ints
# # combo_temp
points2

In [ ]:
# this finds hidden fugas.  
# try to run each of the first set of results above ('points') through this tool, then append the 
# new results to the full DF, and sort again.  
# mark each long pattern with 'has hidden pattern' boolean?  or ?

sample_list = points["Offsets"][4]

hidden_pts = []
n = len(sample_list)
for item in range(3, n):
    list_combinations = list(combinations(sample_list, item))
    for group in list_combinations:
        group_time_ints = numpy.diff(group).tolist()
        hidden_type = classify_by_offset(group_time_ints)
        if 'PEN' in hidden_type:
            print(group)
            print(group_time_ints)
            print(hidden_type)
            hidden_pts.append(group_time_ints)
        if 'ID' in hidden_type:
            print(group)
            print(group_time_ints)
            print(hidden_type)
            hidden_pts.append(group_time_ints)
        

list_combinations

In [ ]:
def classify_entries_as_presentation_types(piece):
    # Classifier with Functions
    points = pd.DataFrame()
    points2 = pd.DataFrame()
    # new_offset_list = []
    nr = piece.getNoteRest()
    det = piece.detailIndex(nr, offset=True)

    # durations and ngrams of durations
    dur = piece.getDuration(df=nr)
    dur_ng = piece.getNgrams(df=dur, n=4)

    # ngrams of melodic entries
    # for chromatic, use:
    # piece.getMelodicEntries(interval_settings=('c', True, True), n=5)
    mel = piece.getMelodicEntries(n=4)
    mels_stacked = mel.stack().to_frame()
    mels_stacked.rename(columns =  {0:"pattern"}, inplace = True)

    # edit distance, based on side-by-side comparison of melodic ngrams
    # gets flexed and other similar soggetti
    dist = piece.getDistance(mel)
    dist_stack = dist.stack().to_frame()


    # filter distances to threshold.  <2 is good
    filtered_dist_stack = dist_stack[dist_stack[0] < 2]
    filtered_dist = filtered_dist_stack.reset_index()
    filtered_dist.rename(columns =  {'level_0':"source", 'level_1':'match'}, inplace = True)

    # Group the filtered distanced patterns
    full_list_of_matches = filtered_dist.groupby('source')['match'].apply(list).reset_index()

    for matches in full_list_of_matches["match"]:
        related_entry_list = mels_stacked[mels_stacked['pattern'].isin(matches)]
        entry_array = related_entry_list.reset_index(level=1).rename(columns = {'level_1': "voice", 0: "pattern"})
        offset_list = entry_array.index.to_list()
        split_list = list(split_by_threshold(offset_list))
        # here is the list of starting offsets of the original set of entries:  slist
        slist = split_list[0]
        temp = temp_dict_of_details(slist, entry_array, det, matches)

        points = points.append(temp, ignore_index=True)
        points['Presentation_Type'] = points['Time_Entry_Intervals'].apply(classify_by_offset)
        points.drop_duplicates(subset=["First_Offset"], keep='first', inplace = True)
        points = points[points['Offsets'].apply(len) > 1]

        l = len(slist)
        if l > 2:
            for r in range(3, l):
    #             list_combinations = list(combinations(slist, r))
                list_combinations = list(combinations(slist, r))
                for slist in list_combinations:

                    temp = temp_dict_of_details(slist, entry_array, det, matches)

                    temp["Presentation_Type"] = classify_by_offset(temp['Time_Entry_Intervals'])

                    if 'PEN' in temp["Presentation_Type"]:
                        points2 = points2.append(temp, ignore_index=True)#.sort_values("First_Offset")
    #                     points = points.append(combo_temp, ignore_index=True).sort_values("First_Offset")
                        points2 = points2[points2['Offsets'].apply(len) > 1]
                    if 'ID' in temp["Presentation_Type"]:
                        points2 = points2.append(combo_temp, ignore_index=True)#.sort_values("First_Offset")
    #                     points = points.append(combo_temp, ignore_index=True).sort_values("First_Offset")
                points2.sort_values("First_Offset")
                points2.drop_duplicates(subset=["First_Offset"], keep='first', inplace = True)

    points_combined = points.append(points2, ignore_index=True).sort_values("First_Offset").reset_index(drop=True)
    points_combined['Flexed_Entries'] = points_combined["Soggetti"].apply(len) > 1
    points_combined["Number_Entries"] = points["Offsets"].apply(len)     
    return points2


In [37]:
# This test works

points = pd.DataFrame()
points2 = pd.DataFrame()
# new_offset_list = []
nr = piece.getNoteRest()
det = piece.detailIndex(nr, offset=True)

# durations and ngrams of durations
dur = piece.getDuration(df=nr)
dur_ng = piece.getNgrams(df=dur, n=4)

# ngrams of melodic entries
# for chromatic, use:
# piece.getMelodicEntries(interval_settings=('c', True, True), n=5)
mel = piece.getMelodicEntries(n=4)
mels_stacked = mel.stack().to_frame()
mels_stacked.rename(columns =  {0:"pattern"}, inplace = True)

# edit distance, based on side-by-side comparison of melodic ngrams
# gets flexed and other similar soggetti
dist = piece.getDistance(mel)
dist_stack = dist.stack().to_frame()


# filter distances to threshold.  <2 is good
filtered_dist_stack = dist_stack[dist_stack[0] < 2]
filtered_dist = filtered_dist_stack.reset_index()
filtered_dist.rename(columns =  {'level_0':"source", 'level_1':'match'}, inplace = True)

# Group the filtered distanced patterns
full_list_of_matches = filtered_dist.groupby('source')['match'].apply(list).reset_index()

for matches in full_list_of_matches["match"]:
    related_entry_list = mels_stacked[mels_stacked['pattern'].isin(matches)]
    entry_array = related_entry_list.reset_index(level=1).rename(columns = {'level_1': "voice", 0: "pattern"})
    offset_list = entry_array.index.to_list()
    split_list = list(split_by_threshold(offset_list))
    # here is the list of starting offsets of the original set of entries:  slist
    slist = split_list[0]
    temp = temp_dict_of_details(slist, entry_array, det, matches)

    points = points.append(temp, ignore_index=True)
    points['Presentation_Type'] = points['Time_Entry_Intervals'].apply(classify_by_offset)
    points.drop_duplicates(subset=["First_Offset"], keep='first', inplace = True)
    points = points[points['Offsets'].apply(len) > 1]

    test = [278.0, 286.0, 294.0, 298.0, 306.0, 310.0]

    l = len(test)  
    for item in range(3, l):
        list_combinations = list(combinations(test, item))
        for group in list_combinations:
            group_time_ints = numpy.diff(group).tolist()
            hidden_type = classify_by_offset(group_time_ints)
            for item in group:
    #         print(item)
                array = group[entry_array.index.get_level_values(0).isin(item)]
                short_offset_list = array.index.to_list()
                time_ints = numpy.diff(array.index).tolist()
                voice_list = array['voice'].to_list()
                if 'PEN' in hidden_type:
                    print(group)
                    print(group_time_ints)
                    print(hidden_type)
                    hidden_pts.append(group_time_ints)
                if 'ID' in hidden_type:
                    print(group)
                    print(group_time_ints)
                    print(hidden_type)
                    hidden_pts.append(group_time_ints)
# len(split_list[0])           

TypeError: only list-like objects are allowed to be passed to isin(), you passed a [float]

In [ ]:
#  This shows how the classifier works:

if len(set(offset_diffs)) == 1 and len(offset_diffs) > 1:
    print('This is a PEN')
    # elif (len(offset_difference_list) %2 != 0) and (len(set(alt_list)) == 1):
elif (len(offset_diffs) % 2 != 0) and (len(set(alt_list)) == 1) and (len(offset_diffs) >= 3):
    print('This is an ID')
elif len(offset_diffs) >= 1:
    print('This is a FUGA')

In [ ]:
# This shows how combinations works for a given set of time intervals
offset_diffs = [12.0, 32.0, 12.0, 4.0]
l = len(offset_diffs)
# print(l)
if l > 2:
    for r in range(3, l):
        print(r)
        list_combinations = list(combinations(offset_diffs, r))
#         for slist in list_combinations:
        print(list_combinations)

In [28]:
slist = [278.0, 286.0, 294.0, 298.0, 306.0, 310.0]
l = len(slist)
# for r in range(3, 6):
list_combinations = list(combinations(slist, 4))
#     for tiny_list in list_combinations:

In [29]:
print(list_combinations)

[(278.0, 286.0, 294.0, 298.0), (278.0, 286.0, 294.0, 306.0), (278.0, 286.0, 294.0, 310.0), (278.0, 286.0, 298.0, 306.0), (278.0, 286.0, 298.0, 310.0), (278.0, 286.0, 306.0, 310.0), (278.0, 294.0, 298.0, 306.0), (278.0, 294.0, 298.0, 310.0), (278.0, 294.0, 306.0, 310.0), (278.0, 298.0, 306.0, 310.0), (286.0, 294.0, 298.0, 306.0), (286.0, 294.0, 298.0, 310.0), (286.0, 294.0, 306.0, 310.0), (286.0, 298.0, 306.0, 310.0), (294.0, 298.0, 306.0, 310.0)]


In [30]:
list_offsets = [294.0, 298.0, 306.0, 310.0]

In [23]:
offset_diffs = [4, 5, 6]

In [24]:
alt_list = offset_diffs[::2]

if len(set(offset_diffs)) == 1 and len(offset_diffs) > 1:
    print('This is a PEN')
    # elif (len(offset_difference_list) %2 != 0) and (len(set(alt_list)) == 1):
elif (len(offset_diffs) % 2 != 0) and (len(set(alt_list)) == 1) and (len(offset_diffs) >= 3):
    print('This is an ID')
elif len(offset_diffs) >= 1:
    print('This is a FUGA')

This is a FUGA
